# IPython 内建魔法命令

参考：[magics](https://ipython.readthedocs.io/en/latest/interactive/magics.html)

```{note}
致Jupyter用户：魔法命令（Magics）是特定于 IPython 内核并由其提供的功能。魔法命令是否在内核中可用，是由内核开发者根据每个内核的具体情况决定的。为了正常工作，魔法命令必须使用底层语言中无效的语法元素。例如，IPython 内核使用 `%` 作为魔法命令的语法元素，因为 `%` 在Python中不是有效的单目运算符。然而，在其他语言中，`%` 可能具有不同的含义。
```

## `%alias`

为系统命令定义别名。

`%alias alias_name cmd` 定义 `alias_name` 作为 `cmd` 的别名。

然后，输入 `alias_name params` 将会执行系统命令 `cmd params`（从你底层的操作系统中执行）。

别名的优先级低于魔法函数和 Python 普通变量，因此如果 `foo` 既是 Python 变量又是一个别名，那么在删除 Python 变量 `del foo` 之前，别名无法被执行。

你可以在别名定义中使用 `%l` 占位符来表示当别名被调用时整行内容。例如：
```ipython
In [2]: alias bracket echo "Input in brackets: <%l>"
In [3]: bracket hello world
Input in brackets: <hello world>
```

你也可以使用 `%s` 占位符（每个参数一个）来定义带有参数的别名：

```python
In [1]: alias parts echo first %s second %s
In [2]: %parts A B
first A second B
In [3]: %parts A
Incorrect number of arguments: 2 expected.
parts is an alias to: 'echo first %s second %s'
```

注意，`%l` 和 `%s` 是互斥的。你只能在别名中使用其中一个。

别名扩展Python变量的方式与使用 `!` 或 `!!` 的系统调用相同：所有以 `$` 开头的表达式都会被扩展。关于语义规则的详细信息，请参阅 [PEP-215](https://peps.python.org/pep-0215/)。这是IPython用于变量扩展的库。如果你想访问真正的shell变量，需要额外加一个 `$` 以防止IPython对其进行扩展：

```python
In [6]: alias show echo
In [7]: PATH='A Python string'
In [8]: show $PATH
A Python string
In [9]: show $$PATH
/usr/local/lf9560/bin:/usr/local/intel/compiler70/ia32/bin:...
```

你可以使用别名功能来访问 `$PATH` 中的所有内容。请参阅 `%rehashx` 函数，它会自动为 `$PATH` 中的内容创建别名。

如果调用时不带参数，`%alias` 会打印当前系统中的别名表。对于posix系统，默认别名包括 `cat`、`cp`、`mv`、`rm`、`rmdir` 和 `mkdir`，以及其他平台特定的别名。对于基于Windows的系统，默认别名包括 `copy`、`ddir`、`echo`、`ls`、`ldir`、`mkdir`、`ren` 和 `rmdir`。

你可以在别名后加上问号来查看其定义：

```python
In [1]: cat?
Repr: <alias cat for 'cat'>
```

## `%alias_magic`

```ipython
%alias_magic [-l] [-c] [-p PARAMS] name target
```

为现有的行魔法或单元魔法创建别名。

示例

```python
In [1]: %alias_magic t timeit
Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.

In [2]: %t -n1 pass
107 ns ± 43.6 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [3]: %%t -n1
   ...: pass
   ...:
107 ns ± 58.3 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [4]: %alias_magic --cell whereami pwd
UsageError: Cell magic function `%%pwd` not found.
In [5]: %alias_magic --line whereami pwd
Created `%whereami` as an alias for `%pwd`.

In [6]: %whereami
Out[6]: '/home/testuser'

In [7]: %alias_magic h history -p "-l 30" --line
Created `%h` as an alias for `%history -l 30`.
```

位置参数：
- `name`：要创建的魔法的名称。
- `target`：现有行魔法或单元魔法的名称。

选项：
- `-l, --line`：创建行魔法别名。
- `-c, --cell`：创建单元魔法别名。
- `-p PARAMS, --params PARAMS`：传递给魔法函数的参数。

## `%autoawait`

允许更改 `autoawait` 选项的状态。

这允许你设置特定的异步代码运行器。

如果不传递任何值，将打印当前使用的异步集成及其是否已激活。

它可以接受以下按顺序评估的值：

- `False/false/off`：禁用 `autoawait` 集成。
- `True/true/on`：激活 `autoawait` 集成，使用配置的默认事件循环。
- `asyncio/curio/trio`：激活 `autoawait` 集成，并使用与指定库的集成。
- `sync`：启用伪同步集成（主要用于不使用真实事件循环运行 IPython 的情况，并禁用运行异步代码。使用伪同步循环启用异步代码的行为未定义，可能会导致 IPython 崩溃。）

如果传递的参数与上述任何值都不匹配，并且是一个 Python 标识符，则从用户命名空间获取该对象并将其设置为运行器，并激活 `autoawait`。

如果对象是一个完全限定的对象名称，尝试导入它并将其设置为运行器，并激活 `autoawait`。

`autoawait` 的确切行为是实验性的，可能会在不同版本的 IPython 和 Python 之间发生变化。

## `%autocall`

使函数无需输入括号即可调用。

用法：

`%autocall [mode]`

模式可以是以下之一：0->关闭，1->智能，2->完全。如果不给定，则切换开启和关闭（记住之前的状态）。

更详细地说，这些值的含义如下：

0 -> 完全禁用

1 -> 激活，但如果行上没有参数则不应用。

在这种模式下，你会得到：

```python
In [1]: callable
Out[1]: <built-in function callable>

In [2]: callable 'hello'
------> callable('hello')
Out[2]: False
```

2 -> 始终激活。即使没有参数，可调用对象也会被调用：

```python
In [2]: float
------> float()
Out[2]: 0.0
```

请注意，即使 `autocall` 关闭，你仍然可以在行首使用 `/` 将命令行上的第一个参数视为函数并为其添加括号：

```python
In [8]: /str 43
------> str(43)
Out[8]: '43'
```


## `%automagic`

使魔法函数无需输入初始的 `%` 即可调用。

不带参数时，切换开启/关闭（关闭时，当然必须调用 `%automagic`）。带参数时，设置该值，你可以使用以下任意值（不区分大小写）：

- `on`, `1`, `True`：激活
- `off`, `0`, `False`：停用。

请注意，魔法函数的优先级最低，因此如果有一个变量与魔法函数的名称冲突，则 `automagic` 对该函数不起作用（你会得到该变量）。但是，如果你删除该变量（`del var`），之前被遮蔽的魔法函数将再次对 `automagic` 可见。

## `%bookmark`

管理 IPython 的书签系统。

```bash
%bookmark <name> - 将书签设置为当前目录
%bookmark <name> <dir> - 将书签设置为 <dir>
%bookmark -l - 列出所有书签
%bookmark -d <name> - 删除书签
%bookmark -r - 删除所有书签
```

你可以在之后通过以下方式访问书签文件夹：

```bash
%cd -b <name>
```

或者如果 `<name>` 不是一个目录名称，并且定义了相应的书签，则可以简单地使用 `%cd <name>`。

你的书签在 IPython 会话之间持久保存，但它们与每个配置文件关联。

## `%cd`

更改当前工作目录。

此命令自动维护你在 IPython 会话期间访问的目录列表，存储在变量 `_dh` 中。命令 `%dhist` 会很好地格式化显示此历史记录。你也可以使用 `cd -<tab>` 方便地查看目录历史记录。

用法：

- `cd 'dir'`：更改为目录 `'dir'`。
- `cd -`：更改为上次访问的目录。
- `cd -<n>`：更改为目录历史记录中的第 n 个目录。
- `cd --foo`：更改为历史记录中与 `'foo'` 匹配的目录。
- `cd -b <bookmark_name>`：跳转到由 `%bookmark` 设置的书签。

在 `cd -b` 后按 Tab 键可以自动补全书签名。

注意：

如果存在名为 `<bookmark_name>` 的书签，但不存在名为 `<bookmark_name>` 的目录，则 `cd <bookmark_name>` 就足够了。

选项：

- `-q`：静默模式。执行 `cd` 命令后不打印工作目录。默认情况下，IPython 的 `cd` 命令会打印此目录，因为默认提示符不显示路径信息。

注意：

请注意，`!cd` 不适用于此目的，因为在执行 `!command` 后，运行命令的 shell 会立即被丢弃。

示例：

```python
In [10]: cd parent/child
/home/tsuser/parent/child
```

## `%env`

获取、设置或列出环境变量。

用法：

- `%env`：列出所有环境变量/值。
- `%env var`：获取变量 `var` 的值。
- `%env var val`：设置变量 `var` 的值为 `val`。
- `%env var=val`：设置变量 `var` 的值为 `val`。
- `%env var=$val`：设置变量 `var` 的值，尽可能使用 Python 扩展。

## `%gui`

启用或禁用 IPython GUI 事件循环集成。

```bash
%gui [GUINAME]
```

此魔法取代了使用（`pylab/wthread/etc.`）命令行标志激活的 IPython 线程化 shell。现在可以在运行时启用 GUI 工具包，并且键盘中断应该可以正常工作，没有任何问题。支持以下工具包：wxPython、PyQt4、PyGTK、Tk 和 Cocoa（OSX）：

```bash
%gui wx      # 启用 wxPython 事件循环集成
%gui qt      # 启用 PyQt/PySide 事件循环集成，使用最新版本。
%gui qt6     # 启用 PyQt6/PySide6 事件循环集成
%gui qt5     # 启用 PyQt5/PySide2 事件循环集成
%gui gtk     # 启用 PyGTK 事件循环集成
%gui gtk3    # 启用 Gtk3 事件循环集成
%gui gtk4    # 启用 Gtk4 事件循环集成
%gui tk      # 启用 Tk 事件循环集成
%gui osx     # 启用 Cocoa 事件循环集成（需要 %matplotlib 1.1）
%gui         # 禁用所有事件循环集成
```

警告：在调用这些命令后，你可以简单地创建一个应用程序对象，但**不要自己启动事件循环**，因为我们已经处理了它。

## `%history`

```bash
%history [-n] [-o] [-p] [-t] [-f FILENAME] [-g [PATTERN ...]]
             [-l [LIMIT]] [-u]
             [range ...]
```

打印输入历史记录（`_i<n>` 变量），最近的记录在最后。

默认情况下，输入历史记录不带行号打印，以便可以直接粘贴到编辑器中。使用 `-n` 选项显示行号。

默认情况下，显示当前会话的所有输入历史记录。可以使用以下语法指定历史记录的范围：

```bash
4           # 当前会话的第 4 行
4-6         # 当前会话的第 4 到第 6 行
243/1-5     # 会话 243 的第 1 到第 5 行
~2/7        # 当前会话前 2 个会话的第 7 行
~8/1-~6/5   # 从 8 个会话前的第 1 行到 6 个会话前的第 5 行
```

多个范围可以用空格分隔。

`%macro`、`%save`、`%edit`、`%rerun` 使用相同的语法。

示例：

```python
In [6]: %history -n 4-6
4: a = 12
5: print(a**2)
6: %history -n 4-6
```

位置参数：

- `range`

选项：

- `-n`：为每个输入打印行号。此功能仅在使用了编号提示符时可用。
- `-o`：同时打印每个输入的输出。
- `-p`：在每个输入前打印经典的 `>>>` Python 提示符。这对于制作文档以及与 `-o` 结合使用以生成可用于 doctest 的输出非常有用。
- `-t`：打印“翻译”后的历史记录，即 IPython 理解的内容。IPython 会过滤你的输入并将其全部转换为有效的 Python 源代码后再执行（例如，魔法命令或别名会被转换为函数调用）。使用此选项，你将看到本机历史记录，而不是用户输入的版本：`%cd /` 将显示为 `get_ipython().run_line_magic("cd", "/")`，而不是 `%cd /`。
- `-f FILENAME`：将输出重定向到给定的文件，而不是打印到屏幕。文件总是被覆盖，但在可能的情况下，IPython 会首先请求确认。特别是，从 IPython Notebook 界面运行命令 `history -f FILENAME` 会替换 `FILENAME`，即使它已经存在，也不会请求确认。
- `-g <[PATTERN ...]>`：将参数视为在（完整）历史记录中搜索的 glob 模式。这包括保存的历史记录（几乎所有曾经写过的命令）。模式可以包含 `?` 来匹配一个未知字符，`*` 来匹配任意数量的未知字符。使用 `%hist -g` 显示完整的保存历史记录（可能会很长）。
- `-l <[LIMIT]>`：获取所有会话的最后 n 行。指定 n 作为单个参数，默认是最后 10 行。
- `-u`：当使用 `-g` 搜索历史记录时，仅显示唯一的历史记录。

## `%killbgscripts`

杀死由 `%%script` 及其相关命令启动的所有后台进程。

## `%load`

将代码加载到当前前端。

用法：

```bash
%load [options] source
```

其中 `source` 可以是文件名、URL、输入历史记录范围、宏或用户命名空间中的元素。

如果不提供参数，则加载从当前会话开始到此点的历史记录。

选项：

- `-r <lines>`：指定从源文件加载的行或行范围。范围可以指定为 `x-y`（`x..y`）或 Python 风格的 `x:y`（`x..(y-1)`）。`x` 和 `y` 都可以留空（分别表示文件的开头和结尾）。
- `-s <symbols>`：指定从 Python 源文件加载的函数或类。
- `-y`：对于超过 200,000 个字符的源代码，不询问确认。
- `-n`：在搜索源代码时包括用户的命名空间。

此魔法命令可以接受本地文件名、URL、历史记录范围（参见 `%history`）或宏作为参数。除非传递了 `-y` 标志或前端不支持 `raw_input`，否则在加载超过 200,000 个字符的源代码之前会提示确认：

```bash
%load
%load myscript.py
%load 7-27
%load myMacro
%load http://www.example.com/myscript.py
%load -r 5-10 myscript.py
%load -r 10-20,30,40: foo.py
%load -s MyClass,wonder_function myscript.py
%load -n MyClass
%load -n my_module.wonder_function
```

## `%load_ext`

通过模块名称加载 IPython 扩展。

### `%loadpy`

`%load` 的别名。

`%loadpy` 获得了一些灵活性并取消了扩展名的要求，因此它被简化为 `%load`。你可以查看 `%load` 的文档字符串以获取更多信息。

## `%logoff`

暂时停止日志记录。

你必须先前已启动日志记录。

## `%logon`

重新启动日志记录。

此功能用于重新启动你之前使用 `%logoff` 暂时停止的日志记录。要首次启动日志记录，必须使用 `%logstart` 函数，该函数允许你指定可选的日志文件名。

## `%logstart`

在会话中的任何位置启动日志记录。

```bash
%logstart [-o|-r|-t|-q] [log_name [log_mode]]
```

如果不提供名称，则默认在当前目录中以“rotate”模式（见下文）记录到名为 `ipython_log.py` 的文件中。

`%logstart name` 以“backup”模式保存到文件 `name` 中。它保存到该点的历史记录，然后继续记录。

`%logstart` 接受第二个可选参数：日志记录模式。可以是以下之一（注意模式不带引号）：

- `append`：在现有文件的末尾继续记录。
- `backup`：将任何现有文件重命名为 `name~` 并开始记录到 `name`。
- `global`：在你的主目录中追加到单个日志文件。
- `over`：覆盖任何现有日志。
- `rotate`：创建滚动日志：`name.1~`、`name.2~` 等。

选项：

- `-o`：同时记录 IPython 的输出。在此模式下，所有生成 `Out[NN]` 提示符的命令都会记录到日志文件中，紧接在其对应输入行的后面。输出行始终以 `#[Out]# ` 标记开头，以便日志保持有效的 Python 代码。

由于此标记始终相同，因此使用例如简单的 `awk` 调用过滤日志中的输出非常容易：

```bash
awk -F'#\[Out\]# ' '{if($2) {print $2}}' ipython_log.py
```

- `-r`：记录“原始”输入。通常，IPython 的日志包含处理后的输入，因此用户行以最终形式记录，转换为有效的 Python。例如，`%Exit` 记录为 `_ip.run_line_magic("Exit")`。如果给出 `-r` 标志，则所有输入都按输入时的原样记录，不应用任何转换。
- `-t`：在记录的每个输入行前加上时间戳（这些时间戳放在注释中）。
- `-q`：调用日志记录时抑制日志状态消息的输出。

## `%logstate`

打印日志记录系统的状态。

## `%logstop`

完全停止日志记录并关闭日志文件。

要重新开始日志记录，需要进行新的 `%logstart` 调用，可能（但不必须）使用新的文件名、模式和其他选项。

## `%lsmagic`

列出当前可用的魔法函数。

## `%macro`

定义一个宏以便将来重新执行。它接受历史记录范围、文件名或字符串对象。

用法：

```bash
%macro [options] name n1-n2 n3-n4 … n5 .. n6 …
```

选项：

- `-r`：使用“原始”输入。默认情况下，使用“处理后”的历史记录，因此魔法命令以转换为有效 Python 的版本加载。如果给出此选项，则使用在命令行上键入的原始输入。
- `-q`：静默宏定义。默认情况下，会打印一条标记行以指示宏已创建，然后打印宏的内容。如果给出此选项，则在创建宏后不会产生任何打印输出。

这将定义一个名为 `name` 的全局变量，该变量是一个字符串，通过将你指定的输入历史记录切片和行（如上所述的 `n1, n2, …` 数字）连接成一个字符串。该变量像一个自动函数一样工作，当你在提示符下键入 `name` 时，它会像你刚刚键入这些行一样重新执行这些代码。

指定输入范围的语法在 `%history` 中描述。

注意：作为一个“隐藏”功能，你也可以使用传统的 Python 切片表示法，其中 `N:M` 表示从 `N` 到 `M-1` 的数字。

例如，如果你的历史记录包含（使用 `%hist -n` 打印）：

```python
44: x=1
45: y=3
46: z=x+y
47: print(x)
48: a=5
49: print('x',x,'y',y)
```

你可以创建一个包含第 44 到 47 行（包括）和第 49 行的宏，称为 `my_macro`，使用：

```python
In [55]: %macro my_macro 44-47 49
```

现在，键入 `my_macro`（不带引号）将一次性重新执行所有这些代码。

你不需要按顺序给出行号，并且任何给定的行号可以出现多次。你可以从输入历史记录中的任何顺序组装宏。

宏是一个简单的对象，其值存储在 `value` 属性中，但 IPython 的显示系统会检查宏并将其作为代码执行，而不是在你键入其名称时打印它们。

你可以通过显式打印宏来查看其内容：

```python
print(macro_name)
```

## `%magic`

打印有关魔法函数系统的信息。

支持的格式：`-latex`、`-brief`、`-rest`

## `%matplotlib`

```bash
%matplotlib [-l] [gui]
```

设置 matplotlib 以进行交互式工作。

此函数允许你在 IPython 会话的任何阶段激活 matplotlib 的交互式支持。它不会将任何内容导入交互式命名空间。

如果你在使用 IPython Notebook 中的内联 matplotlib 后端，你可以使用以下命令设置启用的图形格式：

```python
In [1]: from matplotlib_inline.backend_inline import set_matplotlib_formats

In [2]: set_matplotlib_formats('pdf', 'svg')
```

内联图形的默认设置为 `'tight'`。这可能会导致显示的图像与使用 `savefig` 创建的相同图像之间存在差异。可以使用以下魔法命令禁用此行为：

```python
In [3]: %config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
```

此外，请参阅 `matplotlib_inline.backend_inline.set_matplotlib_formats` 和 `matplotlib_inline.backend_inline.set_matplotlib_close` 的文档字符串，以获取有关更改内联后端其他行为的更多信息。

示例

要在 IPython Notebook 中启用内联后端：

```python
In [1]: %matplotlib inline
```

在这种情况下，matplotlib 的默认后端是 TkAgg：

```python
In [2]: %matplotlib
Using matplotlib backend: TkAgg
```

但你也可以明确请求不同的 GUI 后端：

```python
In [3]: %matplotlib qt
```

你可以使用 `-l/--list` 选项列出可用的后端：

```python
In [4]: %matplotlib --list
Available matplotlib backends: ['osx', 'qt4', 'qt5', 'gtk3', 'gtk4', 'notebook', 'wx', 'qt', 'nbagg', 'gtk', 'tk', 'inline']
```

位置参数：

- `gui`：要使用的 matplotlib 后端的名称，例如 `'qt'` 或 `'widget'`。如果给出，则使用相应的 matplotlib 后端，否则将使用 matplotlib 的默认后端（你可以在 matplotlib 配置文件中设置）。

选项：

- `-l, --list`：显示可用的 matplotlib 后端。

## `%notebook`

```bash
%notebook filename
```

导出并转换 IPython 笔记本。

此函数可以将当前 IPython 历史记录导出到笔记本文件。例如，要将历史记录导出到 `foo.ipynb`，请执行 `%notebook foo.ipynb`。

位置参数：

- `filename`：笔记本名称或文件名。

## `%page`

```bash
%page [options] OBJECT
```

漂亮地打印对象并通过分页器显示它。

如果不给定对象，则使用 `_`（最后输出）。

选项：

- `-r`：打印 `str(object)`，不进行漂亮打印。

`%pastebin`

```bash
%pastebin [-d “Custom description”][-e 24] 1-7
```

将代码上传到 dpaste.com，并返回 URL。

参数可以是输入历史记录范围、文件名或字符串或宏的名称。

如果不提供参数，则上传从当前会话开始到此点的历史记录。

选项：

- `-d`：传递自定义描述。默认值为“Pasted from IPython”。
- `-e`：传递链接过期的天数。默认值为 7 天。

## `%pdb`

控制自动调用 pdb 交互式调试器的功能。

调用方式为 `%pdb on`、`%pdb 1`、`%pdb off` 或 `%pdb 0`。如果不带参数调用，则作为切换开关。

当触发异常时，IPython 可以选择在回溯打印后调用交互式 pdb 调试器。`%pdb` 切换此功能的开和关。

此功能的初始状态在你的配置文件中设置（选项为 `InteractiveShell.pdb`）。

如果你只想在异常发生后激活调试器，而不必输入 `%pdb on` 并重新运行代码，可以使用 `%debug` 魔法。

## `%pdef`

打印任何可调用对象的调用签名。

如果对象是类，则打印构造函数信息。

示例：

```python
In [3]: %pdef urllib.urlopen
urllib.urlopen(url, data=None, proxies=None)
```

## `%pdoc`

打印对象的文档字符串。

如果给定的对象是类，它将同时打印类和构造函数的文档字符串。

## `%pfile`

打印（或通过分页器运行）定义对象的文件。

文件在对象定义开始的行打开。IPython 将尊重环境变量 `PAGER`（如果已设置），否则将尽力以方便的形式打印文件。

如果给定的参数不是当前定义的对象，IPython 将尝试将其解释为文件名（如果需要，自动添加 `.py` 扩展名）。因此，你可以使用 `%pfile` 作为语法高亮代码查看器。

## `%pinfo`

提供有关对象的详细信息。

`%pinfo object` 只是 `object?` 或 `?object` 的同义词。

## `%pinfo2`

提供有关对象的额外详细信息。

`%pinfo2 object` 只是 `object??` 或 `??object` 的同义词。

## `%pip`

在当前内核中运行 pip 包管理器。

用法：

```bash
%pip install [pkgs]
```

## `%popd`

更改到从堆栈顶部弹出的目录。

## `%pprint`

切换漂亮打印的开/关。

## `%precision`

设置浮点数的漂亮打印精度。

可以设置整数精度或格式字符串。

如果导入了 numpy 并且精度是整数，numpy 显示精度也将通过 `numpy.set_printoptions` 设置。

如果不给定参数，则恢复默认值。

示例：

```python
In [1]: from math import pi

In [2]: %precision 3
Out[2]: '%.3f'

In [3]: pi
Out[3]: 3.142

In [4]: %precision %i
Out[4]: '%i'

In [5]: pi
Out[5]: 3

In [6]: %precision %e
Out[6]: '%e'

In [7]: pi**10
Out[7]: 9.364805e+04

In [8]: %precision
Out[8]: '%r'

In [9]: pi**10
Out[9]: 93648.047476082982
```

## `%prun`

通过 Python 代码分析器运行语句。

用法，行模式：

```bash
%prun [options] statement
```

用法，单元模式：

```bash
%%prun [options] [statement]
code...
code...
```

在单元模式下，额外的代码行会附加到第一行中的语句（可能为空）。单元模式允许你轻松分析多行代码块，而无需将它们放在单独的函数中。

给定的语句（不需要引号）通过类似于 `profile.run()` 函数的 Python 分析器运行。名称空间在内部进行管理以正确工作；`profile.run` 不能在 IPython 中使用，因为它对名称空间做出了某些假设，而这些假设在 IPython 中不成立。

选项：

- `-l <limit>`：你可以对打印的分析内容进行限制。`limit` 值可以是：

  - 字符串：仅打印包含此字符串的函数名的信息。
  - 整数：仅打印这么多行。
  - 浮点数（在 0 和 1 之间）：打印报告的这一部分（例如，使用 `0.4` 的限制仅查看最上面的 40%）。

  你可以通过多次使用该选项来组合多个限制。例如，`-l __init__ -l 5` 将仅打印关于类构造函数的最上面的 5 行信息。

- `-r`：返回由分析生成的 `pstats.Stats` 对象。该对象包含分析的所有信息，你可以稍后使用它进行进一步分析或在其他函数中使用。

- `-s <key>`：按给定的键对分析进行排序。你可以通过多次使用该选项来提供多个键：`-s key1 -s key2 -s key3...`。默认排序键是 `time`。

  以下内容直接复制自 `profile` 文档，参考下文：

  当提供多个键时，当所有前面选择的键相等时，附加键用作次要标准。

  任何键名都可以使用缩写，只要缩写是明确的。以下是目前定义的键：

  | 有效参数 | 含义 |
  | --- | --- |
  | `"calls"` | 调用次数 |
  | `"cumulative"` | 累计时间 |
  | `"file"` | 文件名 |
  | `"module"` | 文件名 |
  | `"pcalls"` | 原始调用次数 |
  | `"line"` | 行号 |
  | `"name"` | 函数名 |
  | `"nfl"` | 名称/文件/行 |
  | `"stdname"` | 标准名称 |
  | `"time"` | 内部时间 |

  请注意，所有基于统计的排序都是降序排列（将最耗时的项放在前面），而名称、文件和行号搜索是升序排列（即按字母顺序）。`"nfl"` 和 `"stdname"` 之间的细微区别在于标准名称是按打印的名称排序的，这意味着嵌入的行号以一种奇怪的方式进行比较。例如，行号 3、20 和 40 将（如果文件名相同）按字符串顺序 `"20"`、`"3"` 和 `"40"` 出现。相比之下，`"nfl"` 对行号进行数字比较。实际上，`sort_stats("nfl")` 等同于 `sort_stats("name", "file", "line")`。

- `-T <filename>`：将屏幕上显示的分析结果保存到文本文件中。分析仍会显示在屏幕上。

- `-D <filename>`：将分析统计数据保存（通过 `dump_stats`）到给定的文件名。该数据采用 `pstats` 模块理解的格式，并通过调用分析对象的 `dump_stats()` 方法生成。分析仍会显示在屏幕上。

- `-q`：抑制输出到分页器。最好与上面的 `-T` 和/或 `-D` 一起使用。

如果你想在分析器的控制下运行完整的程序，请使用 `%run -p [prof_opts] filename.py [args to program]`，其中 `prof_opts` 包含此处描述的分析器特定选项。

你可以通过以下方式阅读 `profile` 模块的完整文档：

```python
In [1]: import profile; profile.help()
```

版本 7.3 中的更改：用户变量不再展开，魔法行始终保持不变。

## `%psearch`

在命名空间中按通配符搜索对象。

```bash
%psearch [options] PATTERN [OBJECT TYPE]
```

注意：`?` 可以用作 `%psearch` 的同义词，无论是在开头还是结尾：`a*?` 和 `?a*` 都等同于 `%psearch a*`。尽管如此，命令行的其余部分必须保持不变（选项在前），因此例如以下形式是等价的：

```bash
%psearch -i a* function
-i a* function?
?-i a* function
```

参数：

- `PATTERN`：一个包含 `*` 作为通配符的字符串，类似于其在 shell 中的使用。该模式在搜索路径中的所有命名空间中匹配。默认情况下，以单个 `_` 开头的对象不匹配，许多 IPython 生成的对象都有一个单下划线。默认情况下，匹配是大小写不敏感的。匹配也针对对象的属性进行，而不仅仅是模块中的对象。

- `[OBJECT TYPE]`：是 `types` 模块中的 Python 类型的名称。名称以小写形式给出，不带结尾的 `type`，例如 `StringType` 写为 `string`。通过添加类型，仅匹配给定类型的对象。使用 `all` 使模式匹配所有类型（这是默认值）。

选项：

- `-a`：使模式匹配名称以单个下划线开头的对象。这些名称通常从搜索中省略。

- `-i/-c`：使模式大小写不敏感/敏感。如果不给出这些选项，默认值从配置文件中读取，选项为 `InteractiveShell.wildcards_case_sensitive`。如果此选项未在配置文件中指定，IPython 的内部默认值是进行大小写敏感的搜索。

- `-e/-s NAMESPACE`：排除/搜索给定的命名空间。你可以指定在以下命名空间中搜索模式：`'builtin'`、`'user'`、`'user_global'`、`'internal'`、`'alias'`，其中 `'builtin'` 和 `'user'` 是搜索的默认值。请注意，指定命名空间时不应使用引号。

- `-l`：列出所有可用于对象匹配的对象类型。此功能可以在没有参数的情况下使用。

  `'Builtin'` 包含 Python 模块 `builtin`，`'user'` 包含所有用户数据，`'alias'` 仅包含 shell 别名，没有 Python 对象，`'internal'` 包含 IPython 使用的对象。`'user_global'` 命名空间仅由嵌入式 IPython 实例使用，并且包含模块级全局变量。你可以使用 `-s` 添加命名空间或使用 `-e` 排除它们（这些选项可以多次给出）。

示例：

```bash
%psearch a*            -> 以 'a' 开头的对象
%psearch -e builtin a* -> 不在 'builtin' 命名空间中且以 'a' 开头的对象
%psearch a* function   -> 所有以 'a' 开头的函数
%psearch re.e*         -> 模块 're' 中以 'e' 开头的对象
%psearch r*.e*         -> 模块名以 'r' 开头且对象名以 'e' 开头的对象
%psearch r*.* string   -> 模块名以 'r' 开头的所有字符串
```

大小写敏感的搜索：

```bash
%psearch -c a*         -> 列出所有以小写 'a' 开头的对象
```

显示以单个 `_` 开头的对象：

```bash
%psearch -a _*         -> 列出以单个下划线开头的对象
```

列出可用对象：

```bash
%psearch -l            -> 列出所有可用的对象类型
```

## `%psource`

打印（或通过分页器运行）对象的源代码。

## `%pushd`

将当前目录放在堆栈上并更改目录。

用法：

```bash
%pushd [‘dirname’]
```

## `%pwd`

返回当前工作目录路径。

示例：

```python
In [9]: pwd
Out[9]: '/home/tsuser/sprint/ipython'
```

## `%pycat`

通过分页器显示语法高亮的文件。

此魔法类似于 `cat` 实用程序，但它假设文件是 Python 源代码，并显示语法高亮。

此魔法命令可以接受本地文件名、URL、历史记录范围（参见 `%history`）或宏作为参数。

如果不给定参数，则打印当前会话到此点的历史记录。

```bash
%pycat myscript.py
%pycat 7-27
%pycat myMacro
%pycat http://www.example.com/myscript.py
```

## `%pylab`

```bash
%pylab [--no-import-all] [gui]
```

加载 numpy 和 matplotlib 以进行交互式工作。

此函数允许你在 IPython 会话的任何阶段激活 pylab（matplotlib、numpy 和交互支持）。

`%pylab` 进行以下导入：

```python
import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
np = numpy
plt = pyplot

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *
```

如果你传递 `--no-import-all`，最后两个导入将被排除。

有关在不影响交互命名空间的情况下激活 matplotlib 的更多详细信息，请参阅 `%matplotlib` 魔法。

位置参数：

- `gui`：要使用的 matplotlib 后端的名称，例如 `'qt'` 或 `'widget'`。如果给出，则使用相应的 matplotlib 后端，否则将使用 matplotlib 的默认后端（你可以在 matplotlib 配置文件中设置）。

选项：

- `--no-import-all`：防止 IPython 执行 `import *` 到交互命名空间。你可以通过 `InteractiveShellApp.pylab_import_all` 可配置项来控制此标志的默认行为。

## `%quickref`

显示快速参考表。

## `%recall`

重复命令，或将命令放入输入行以进行编辑。

`%recall` 和 `%rep` 是等效的。

`%recall`（无参数）：

将最后一个计算结果（存储在特殊变量 `_` 中）的字符串版本放置到下一个输入提示符。允许你创建不需要使用复制粘贴的复杂命令行：

```python
In[1]: l = ["hei", "vaan"]
In[2]: "".join(l)
Out[2]: heivaan
In[3]: %recall
In[4]: heivaan_ <== 光标闪烁
```

`%recall 45`：

将历史记录行 45 放置到下一个输入提示符。使用 `%hist` 查找行号。

`%recall 1-4`：

将指定行合并为一个单元格，并将其放置到下一个输入提示符。请参阅 `%history` 的切片语法。

`%recall foo+bar`：

如果 `foo+bar` 可以在用户命名空间中求值，则结果将放置到下一个输入提示符。否则，搜索历史记录中包含该子字符串的行，并将最近的一行放置到下一个输入提示符。

## `%rehashx`

使用 `$PATH` 中的所有可执行文件更新别名表。

`rehashx` 显式检查 `$PATH` 中的每个条目是否是具有执行权限（`os.X_OK`）的文件。

在 Windows 上，它检查可执行性，方法是与存储在 IPython 配置变量 `win_exec_ext` 中的 `|` 分隔的扩展名字符串进行匹配。默认值为 `'exe|com|bat'`。

此函数还会重置模块补全器的根模块缓存，用于慢速文件系统。

## `%reload_ext`

通过模块名称重新加载 IPython 扩展。

## `%rerun`

重新运行之前的输入。

默认情况下，你可以指定要重复的输入历史记录范围（如 `%history`）。如果不带参数调用，它将重复最后一行。

选项：

- `-l <n>`：重复最后 n 行输入，不包括当前命令。
- `-g foo`：重复包含 `foo` 的最近一行。

## `%reset`

通过删除用户定义的所有名称来重置命名空间，如果不带参数调用，或者通过删除某些类型的对象来重置命名空间，例如当前在 IPython 的 `In[]` 和 `Out[]` 容器中的所有内容（详见参数）。

参数：

- `-f`：强制重置而不询问确认。
- `-s`：“软”重置：仅清除你的命名空间，保留历史记录不变。对象的引用可能会保留。默认情况下（不使用此选项），我们进行“硬”重置，为你提供一个新的会话，并删除当前会话中所有对象的引用。
- `--aggressive`：尝试从 `sys.modules` 中积极删除模块；这可能允许你重新导入已更新的 Python 模块并获取更改，但可能会产生意外后果。
- `in`：重置输入历史记录。
- `out`：重置输出历史记录。
- `dhist`：重置目录历史记录。
- `array`：仅重置 NumPy 数组变量。

另请参阅：

`reset_selective`：以 `%reset_selective` 调用。

示例：

```python
In [6]: a = 1
In [7]: a
Out[7]: 1
In [8]: 'a' in get_ipython().user_ns
Out[8]: True
In [9]: %reset -f
In [1]: 'a' in get_ipython().user_ns
Out[1]: False
In [2]: %reset -f in
Flushing input history
In [3]: %reset -f dhist in
Flushing directory history
Flushing input history
```

注意：

从不实现标准输入的客户端（如 ipython notebook 接口）调用此魔法时，将重置命名空间而不进行确认。

## `%reset_selective`

通过删除用户定义的名称来重置命名空间。

输入/输出历史记录保留在原处，以防你需要它们。

```bash
%reset_selective [-f] regex
```

如果不包含正则表达式，则不执行任何操作。

选项：

- `-f`：强制重置而不询问确认。

另请参阅：

`reset`：以 `%reset` 调用。

示例：

我们首先完全重置命名空间，以便你的输出与本示例相同，出于教学原因；实际上你不需要完全重置：

```python
In [1]: %reset -f
```

现在，使用干净的命名空间，我们可以创建一些变量，并使用 `%reset_selective` 仅删除与我们的正则表达式匹配的名称：

```python
In [2]: a=1; b=2; c=3; b1m=4; b2m=5; b3m=6; b4m=7; b2s=8
In [3]: who_ls
Out[3]: ['a', 'b', 'b1m', 'b2m', 'b2s', 'b3m', 'b4m', 'c']
In [4]: %reset_selective -f b[2-3]m
In [5]: who_ls
Out[5]: ['a', 'b', 'b1m', 'b2s', 'b4m', 'c']
In [6]: %reset_selective -f d
In [7]: who_ls
Out[7]: ['a', 'b', 'b1m', 'b2s', 'b4m', 'c']
In [8]: %reset_selective -f c
In [9]: who_ls
Out[9]: ['a', 'b', 'b1m', 'b2s', 'b4m']
In [10]: %reset_selective -f b
In [11]: who_ls
Out[11]: ['a']
```

注意：

从不实现标准输入的客户端（如 ipython notebook 接口）调用此魔法时，将重置命名空间而不进行确认。

## `%run`

在 IPython 中运行指定的文件作为程序。

用法：

```bash
%run [-n -i -e -G]
     [( -t [-N<N>] | -d [-b<N>] | -p [profile options] )]
     ( -m mod | filename ) [args]
```

`filename` 参数应该是纯 Python 脚本（扩展名为 `.py`），或者包含自定义 IPython 语法（如魔法命令）的文件。如果是后者，文件可以是扩展名为 `.ipy` 的脚本，或者是扩展名为 `.ipynb` 的 Jupyter 笔记本。当运行 Jupyter 笔记本时，来自 `print` 语句和其他显示对象的输出将出现在终端中（即使使用终端兼容的后端，matplotlib 图形也会打开）。请注意，在系统命令行中，`jupyter run` 命令提供了类似的功能（尽管目前支持的选项有所不同）。

文件名后的参数将作为命令行参数传递给程序（放入 `sys.argv` 中）。然后，控制权返回到 IPython 提示符。

这类似于在系统提示符下运行 `python file args`，但具有以下优势：提供 IPython 的回溯，并将所有变量加载到交互式命名空间中以供进一步使用（除非使用 `-p`，详见下文）。

文件在最初仅包含 `__name__` 和 `sys.argv` 的命名空间中执行。因此，它将其环境视为如果它正在作为独立程序运行（除了共享全局对象，如之前导入的模块）。但在执行后，IPython 交互式命名空间会更新为程序中定义的所有变量（除了 `__name__` 和 `sys.argv`）。这允许非常方便地加载代码以进行交互式工作，同时为每个程序提供“干净的纸张”来运行。

参数使用类似 shell 的 glob 匹配进行扩展。可以使用模式 `*`、`?`、`[seq]` 和 `[!seq]`。此外，波浪号 `~` 将扩展为用户的主目录。与真正的 shell 不同，引号不会抑制扩展。使用两个反斜杠（例如 `\\*`）来抑制扩展。要完全禁用这些扩展，可以使用 `-G` 标志。

在 Windows 系统上，指定文件时不支持使用单引号。请使用双引号 `"`。

选项：

- `-n`：不将 `__name__` 设置为 `'__main__'`，而是设置为运行文件的名称（不带扩展名，如 Python 在导入时所做的那样）。这允许运行脚本并重新加载其中的定义，而不会调用受 `if __name__ == "__main__"` 子句保护的代码。

- `-i`：在 IPython 的命名空间中运行文件，而不是在空的命名空间中。如果你正在使用在文本编辑器中编写的代码进行实验，并且该代码依赖于交互式定义的变量，这将非常有用。

- `-e`：忽略脚本中调用的 `sys.exit()` 或 `SystemExit` 异常。如果你使用 IPython 运行单元测试，这特别有用，因为单元测试总是以 `sys.exit()` 调用退出。在这种情况下，你感兴趣的是测试结果的输出，而不是看到单元测试模块的回溯。

- `-t`：在运行结束时打印时间信息。IPython 将为你提供脚本的估计 CPU 时间消耗，在 Unix 上使用 `resource` 模块来避免 `time.clock()` 的回绕问题。在 Unix 上，还会给出系统任务所花费时间的估计（对于 Windows 平台，这报告为 `0.0`）。

如果给出 `-t`，可以额外指定 `-N<N>` 选项，其中 `<N>` 必须是一个整数，表示你希望脚本运行的次数。最终的时间报告将包括总时间和每次运行的结果。

例如（测试脚本 `uniq_stable.py`）：

```python
In [1]: run -t uniq_stable

IPython CPU timings (estimated):
  User  :    0.19597 s.
  System:        0.0 s.

In [2]: run -t -N5 uniq_stable

IPython CPU timings (estimated):
Total runs performed: 5
  Times :      Total       Per run
  User  :   0.910862 s,  0.1821724 s.
  System:        0.0 s,        0.0 s.
```

- `-d`：在你的程序下使用 `pdb`（Python 调试器）运行。这允许你逐步执行程序，观察变量等。在内部，IPython 所做的工作类似于调用：

```python
pdb.run('execfile("YOURFILENAME")')
```

并在文件的第一行设置一个断点。你可以使用 `-bN` 选项（其中 `N` 必须是一个整数）来将此自动断点设置为第 `N` 行。例如：

```python
%run -d -b40 myscript
```

将在 `myscript.py` 的第 40 行设置第一个断点。请注意，第一个断点必须设置在实际执行某些操作的行上（不是注释或文档字符串），以便它停止执行。

或者，你可以在不同的文件中指定断点：

```python
%run -d -b myotherfile.py:20 myscript
```

当 `pdb` 调试器启动时，你将看到一个 `(Pdb)` 提示符。你必须首先输入 `'c'`（不带引号）以开始执行，直到第一个断点。

输入 `help` 可以获取调试器的使用信息。你可以通过在提示符下输入 `"import pdb; pdb.help()"` 轻松查看 `pdb` 的完整文档。

- `-p`：在 Python 分析器模块的控制下运行程序（该模块打印详细的执行时间、函数调用等报告）。

你可以在 `-p` 后传递其他选项，这些选项会影响分析器本身的行为。有关详细信息，请参阅 `%prun` 的文档。

在此模式下，程序的变量不会传播回 IPython 交互式命名空间（因为它们保留在分析器执行它们的命名空间中）。

在内部，这会触发对 `%prun` 的调用，有关专门用于分析的选项的详细信息，请参阅其文档。

有一个特殊用法，上述文本不适用：如果文件名以 `.ipy[nb]` 结尾，文件将作为 IPython 脚本运行，就像命令写在 IPython 提示符上一样。

- `-m`：指定要加载的模块名称，而不是脚本路径。类似于 Python 解释器的 `-m` 选项。如果你想与其他 `%run` 选项结合使用，请将此选项放在最后。与 Python 解释器不同，只允许源模块，不允许 `.pyc` 或 `.pyo` 文件。例如：

```python
%run -m example
```

将运行 `example` 模块。

- `-G`：禁用类似 shell 的参数 glob 扩展。

## `%save`

将一组行或宏保存到指定文件名。

用法：

```bash
%save [options] filename [history]
```

选项：

- `-r`：使用“原始”输入。默认情况下，使用“处理后”的历史记录，因此魔法命令以转换为有效 Python 的版本加载。如果给出此选项，则使用在命令行上键入的原始输入。

- `-f`：强制覆盖。如果文件存在，`%save` 将提示覆盖，除非给出 `-f`。

- `-a`：追加到文件而不是覆盖它。

`history` 参数使用与 `%history` 相同的语法来指定输入范围，然后将这些行保存到你指定的文件名中。

如果不指定范围，则保存当前会话到此点的历史记录。

如果文件名没有扩展名，它会自动添加 `.py` 扩展名，并且它会要求确认在覆盖现有文件之前。

如果使用 `-r` 选项，默认扩展名是 `.ipy`。

## `%sc`

Shell 捕获 - 运行 shell 命令并捕获输出（已弃用，请使用 `!`）。

已弃用。次优，保留以实现向后兼容。

你应该使用 `var = !command` 形式代替。例如：

“`%sc -l myfiles = ls ~`” 现在应写为

“`myfiles = !ls ~`”

`myfiles.s`、`myfiles.l` 和 `myfiles.n` 仍然适用，如下面所述。

```bash
%sc [options] varname=command
```

IPython 将使用 `commands.getoutput()` 运行给定的命令，然后将调用的值存储在名为 `varname` 的变量中，并更新用户的交互式命名空间。你的命令可以包含 shell 通配符、管道等。

语法中的 `=` 符号是强制性的，你提供的变量名必须遵循 Python 的标准命名约定。

（存在一种不带变量名的特殊格式，用于内部使用）

选项：

- `-l`：列表输出。在将输出分配给给定变量之前，将其按换行符拆分为列表。默认情况下，输出存储为单个字符串。

- `-v`：详细模式。打印变量的内容。

在大多数情况下，你不需要将输出拆分为列表，因为返回值是一种特殊类型的字符串，可以自动提供其内容，无论是作为列表（按换行符拆分）还是作为空格分隔的字符串。这些在顺序处理或传递给 shell 命令时非常方便。

例如：

```python
# 捕获到变量 a
In [1]: sc a=ls *py

# a 是一个带有嵌入换行符的字符串
In [2]: a
Out[2]: 'setup.py\nwin32_manual_post_install.py'

# 可以将其视为列表：
In [3]: a.l
Out[3]: ['setup.py', 'win32_manual_post_install.py']

# 或作为空格分隔的字符串：
In [4]: a.s
Out[4]: 'setup.py win32_manual_post_install.py'

# a.s 可用于作为单个命令行传递：
In [5]: !wc -l $a.s
  146 setup.py
  130 win32_manual_post_install.py
  276 total

# 而列表形式可用于循环：
In [6]: for f in a.l:
   ...:      !wc -l $f
   ...:
146 setup.py
130 win32_manual_post_install.py
```

类似地，`-l` 选项返回的列表也是特殊的，因为你可以同样调用它们的 `.s` 属性，自动从其内容中获取空格分隔的字符串：

```python
In [7]: sc -l b=ls *py

In [8]: b
Out[8]: ['setup.py', 'win32_manual_post_install.py']

In [9]: b.s
Out[9]: 'setup.py win32_manual_post_install.py'
```

总之，用于输出捕获的列表和字符串具有以下特殊属性：

- `.l`（或 `.list`）：值作为列表。
- `.n`（或 `.nlstr`）：值作为换行符分隔的字符串。
- `.s`（或 `.spstr`）：值作为空格分隔的字符串。

## `%set_env`
设置环境变量。假设“val”是用户命名空间中的一个名称，或者val是评估为字符串的内容。

用法：
- `%set_env var val`： 为var设置值
- `%set_env var=val`：为var设置值
- `%set_env var=$val`：为var设置值，如果可能，使用Python扩展

## `%sx`
Shell 执行 - 运行 shell 命令并捕获输出（`!!` 是简写）。

`%sx command`

IPython 将使用 `commands.getoutput()` 运行给定的命令，并将结果格式化为列表（以 `\n` 分割）。由于输出是 _返回_ 的，它将存储在 IPython 的常规输出缓存 `Out[N]` 和自动变量 `_N` 中。

注意：

1) 如果输入行以 `!!` 开头，则 `%sx` 会自动调用。也就是说，虽然：

`!ls`
会导致 IPython 简单地执行 `system('ls')`，但输入：

`!!ls`
是以下内容的简写：

`%sx ls`
2) `%sx` 与 `%sc` 的不同之处在于 `%sx` 会自动分割成列表，类似于 `%sc -l`。这样做的目的是尽可能方便地通过进一步的 Python 命令处理面向行的 shell 输出。`%sc` 旨在提供更精细的控制，但需要更多的输入。

3) 就像 `%sc -l` 一样，这是一个具有特殊属性的列表：

`.l`（或 `.list`）：值作为列表。
`.n`（或 `.nlstr`）：值作为换行符分隔的字符串。
`.s`（或 `.spstr`）：值作为空白分隔的字符串。
这在尝试将此类列表用作系统命令的参数时非常有用。

## `%system`
Shell 执行 - 运行 shell 命令并捕获输出（!! 是简写）。

%sx 命令

IPython 将使用 `commands.getoutput()` 运行给定的命令，并返回结果格式化为一个列表（按换行符分割）。由于输出是 _返回_ 的，它将被存储在 IPython 的常规输出缓存 `Out[N]` 和自动变量 `_N` 中。

注意：

1) 如果输入行以 `!!` 开头，则 `%sx` 会自动调用。也就是说，虽然：

   ```
   !ls
   ```

   会导致 IPython 简单地执行 `system('ls')`，但输入：

   ```
   !!ls
   ```

   是以下内容的简写：

   ```
   %sx ls
   ```

2) `%sx` 与 `%sc` 的区别在于 `%sx` 会自动分割成一个列表，类似于 `%sc -l`。这样做的目的是尽可能方便地通过进一步的 Python 命令处理面向行的 shell 输出。`%sc` 旨在提供更精细的控制，但需要更多的输入。

3) 就像 `%sc -l` 一样，这是一个具有特殊属性的列表：

   `.l`（或 `.list`）：值作为列表。
   `.n`（或 `.nlstr`）：值作为换行符分隔的字符串。
   `.s`（或 `.spstr`）：值作为空白符分隔的字符串。

   这在尝试将此类列表用作系统命令的参数时非常有用。

## `%tb`

打印最后的回溯信息。

可选地，指定一个异常报告模式，调整回溯信息的详细程度。默认情况下，使用当前激活的异常模式。请参阅 `%xmode` 以更改异常报告模式。

有效模式：Plain（简单）、Context（上下文）、Verbose（详细）和 Minimal（最小）。

## `%time`
计时执行一个 Python 语句或表达式。

将打印 CPU 时间和墙钟时间，并返回表达式的值（如果有）。请注意，在 Win32 系统下，系统时间始终报告为 0，因为它无法测量。

此函数可以用作行魔法和单元魔法：

- 在行模式下，您可以计时单行语句（尽管可以通过使用分号连接多个语句）。
- 在单元模式下，您可以计时单元体（直接跟随的语句会引发错误）。

此函数提供了非常基本的计时功能。如果需要更精细的控制，请使用 `timeit` 魔法。

**版本 7.3 中的更改**：用户变量不再展开，魔法行始终保持未修改状态。

**示例**

```python
In [1]: %time 2**128
CPU times: user 0.00 s, sys: 0.00 s, total: 0.00 s
Wall time: 0.00
Out[1]: 340282366920938463463374607431768211456L

In [2]: n = 1000000

In [3]: %time sum(range(n))
CPU times: user 1.20 s, sys: 0.05 s, total: 1.25 s
Wall time: 1.37
Out[3]: 499999500000L

In [4]: %time print('hello world')
hello world
CPU times: user 0.00 s, sys: 0.00 s, total: 0.00 s
Wall time: 0.00
```

````{note}
如果 Python 编译给定表达式所需的时间超过 0.1 秒，则将报告该时间。

在下面的示例中，实际的指数运算在编译时由 Python 完成，因此尽管表达式可能需要花费大量时间来计算，但该时间纯粹是由于编译导致的：

```python
In [5]: %time 3**9999;
CPU times: user 0.00 s, sys: 0.00 s, total: 0.00 s
Wall time: 0.00 s

In [6]: %time 3**999999;
CPU times: user 0.00 s, sys: 0.00 s, total: 0.00 s
Wall time: 0.00 s
Compiler : 0.78 s
```
````

## `%timeit`
用于执行 Python 语句或表达式的时间测量

使用方法，行模式：
```bash
%timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] 语句
```
或单元格模式：
```bash
%%timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] 初始化代码

代码

代码…
```
使用 timeit 模块来测量 Python 语句或表达式的执行时间。该函数可以作为行魔法和单元格魔法使用：

在行模式下，你可以测量单行语句的执行时间（尽管可以通过使用分号连接多个语句）。

在单元格模式下，第一行的语句用作初始化代码（执行但不计入时间），而单元格的主体部分会被计时。单元格主体可以访问在初始化代码中创建的任何变量。

选项：

`-n<N>`：在循环中执行给定的语句 `<N>` 次。如果未提供 `<N>`，则 `<N>` 会根据需要确定以获得足够的精度。

`-r<R>`：重复执行 `<R>` 次，每次包含 `<N>` 次循环，并取平均结果。默认值：`7`

`-t`：使用 `time.time` 来测量时间，这是 Unix 系统上的默认设置。该函数测量的是挂钟时间（wall time）。

`-c`：使用 `time.clock` 来测量时间，这是 Windows 系统上的默认设置，测量的是挂钟时间。在 Unix 系统上，会使用 `resource.getrusage` 来代替，并返回 CPU 用户时间。

`-p<P>`：使用 `<P>` 位精度来显示时间测量结果。默认值：3

`-q`：静默模式，不打印结果。

`-o`：返回一个 `TimeitResult` 对象，可以将其存储在变量中以更详细地检查结果。

在版本 7.3 中的更改：用户变量不再展开，魔法命令行始终保持不变。

示例

```python
In [1]: %timeit pass
8.26 ns ± 0.12 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)

In [2]: u = None

In [3]: %timeit u is None
29.9 ns ± 0.643 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

In [4]: %timeit -r 4 u == None

In [5]: import time

In [6]: %timeit -n1 time.sleep(2)
```

使用 `%timeit` 报告的时间会比使用 `timeit.py` 脚本报告的时间稍高，尤其是在访问变量时。这是因为 `%timeit` 在 shell 的命名空间中执行语句，而 `timeit.py` 使用单个 setup 语句来导入函数或创建变量。通常，只要不将 `timeit.py` 的结果与 `%timeit` 的结果混合使用，这种偏差并不重要。

## `%%bash`
`%%bash` 脚本魔法

在子进程中使用 bash 运行单元格。

这是 `%%script bash` 的快捷方式。

## `%%capture`
```ipython
%capture [--no-stderr] [--no-stdout] [--no-display] [output]
```
运行单元格，捕获标准输出（stdout）、标准错误（stderr）以及 IPython 的富显示（display()）调用。

位置参数：
output 存储输出的变量名称。这是一个 `utils.io.CapturedIO` 对象，具有 `stdout` 和 `stderr` 属性，用于捕获的输出文本。`CapturedOutput` 还具有 `show()` 方法，用于显示输出，并且支持 `__call__`，因此你可以使用它快速显示输出。如果未指定，捕获的输出将被丢弃。

选项：
- `--no-stderr`: 不捕获标准错误（stderr）。
- `--no-stdout`: 不捕获标准输出（stdout）。
- `--no-display`: 不捕获 IPython 的富显示（`display`）。